In [3]:
import sys, os
import theano
import theano.tensor as T
import numpy as np
import pandas as pd
import lasagne
from sklearn import cross_validation, metrics


In [4]:
#Load the training set and do normalization and re-shaping 
train = pd.read_csv('train.csv').dropna().values
tr_X = train[:,1:].reshape(-1,1,28,28).astype(theano.config.floatX)
tr_X = tr_X/255
ty_Y = train[:,0]

In [6]:
#Split the training data into training and validation sets 
X_train,X_val,y_train,y_val = cross_validation.train_test_split(tr_X,ty_Y, test_size=0.1)


In [19]:
# Convolution Neural Network Model
# Build the convolutional network model 
#   Input           -> 1 x 28 x 28
#   Conv2D          -> 32 x 5 x 5
#   maxPool2D       -> 2x2
#   Dropout         -> p = 0.5
#   Conv2D          -> 32 x 5 x 5
#   maxPool2D       -> 2x2
#   Dropout         -> p = 0.5
#   Full Connected  -> 1024
#   Full Connected  -> 256
#   Full Connected  -> 10 (softmax)
#
#   cost function : categorical cross entropy
#   regularization: l2

def model():

    X = T.tensor4('X')
    y = T.ivector('y')

    net = lasagne.layers.InputLayer(shape=(None,1,28,28), input_var=X)
    net = lasagne.layers.Conv2DLayer(net, num_filters=32, filter_size=(5,5), nonlinearity = lasagne.nonlinearities.rectify,W=lasagne.init.GlorotUniform())
    net = lasagne.layers.MaxPool2DLayer(net, pool_size=(2,2))
    net = lasagne.layers.Conv2DLayer(lasagne.layers.DropoutLayer(net), num_filters=32, filter_size=(5,5), nonlinearity = lasagne.nonlinearities.rectify,W=lasagne.init.GlorotUniform())
    net = lasagne.layers.MaxPool2DLayer(net, pool_size=(2,2))
    net = lasagne.layers.DenseLayer(lasagne.layers.DropoutLayer(net), num_units=1024, nonlinearity = lasagne.nonlinearities.rectify)
    net = lasagne.layers.DenseLayer(net, num_units=256, nonlinearity = lasagne.nonlinearities.rectify)
    net = lasagne.layers.DenseLayer(net, num_units=10, nonlinearity = lasagne.nonlinearities.softmax)

    params = lasagne.layers.get_all_params(net, trainable=True)
    prediction = lasagne.layers.get_output(net)
    cost = lasagne.objectives.categorical_crossentropy(prediction,y)
    cost = cost.mean()

    updates = lasagne.updates.adagrad(cost,params, learning_rate=0.01)
    

    true_pred = lasagne.layers.get_output(net, deterministic=True)
    y_pred = T.argmax(true_pred, axis = 1)

    train = theano.function([X,y], [cost,prediction], updates = updates, allow_input_downcast=True)
    predict= theano.function([X], y_pred, allow_input_downcast=True)
    return train,predict

In [20]:
#Helper function to iterate over mini batches
def batch(X,y,n=128, randomize=True):
    l = len(X)
    if randomize:
        perm = np.random.permutation(l)
        Xrand = X[perm]
        yrand = y[perm]
    else:
        Xrand = X
        yrand = y
    for i in range(0,l,n):
        yield Xrand[i:min(i+n,l)],yrand[i:min(i+n,l)]



In [21]:
#create the network
fit,predict = model()


In [36]:

#training
for i in range(100):
    for X_batch,y_batch in batch(X_train,y_train, n=256):
        cost,pred = fit(X_batch,y_batch)
    #print 'Epoch {0}/{1} : cost : {2}'.format(i,100,cost)
    print i,cost,
    sys.stdout.flush()

0 0.00165545730852 1 0.0029792143032 2 0.00696317432448 3 0.000286816153675 4 0.00751735363156 5 0.002448995132 6 0.00530381640419 7 0.00403619091958 8 0.00164288445376 9 0.00237629073672 10 0.00166937662289 11 0.00155769917183 12 0.0122303254902 13 0.00578951276839 14 0.00305026816204 15 0.0300751198083 16 0.00993761327118 17 0.00151582737453 18 0.00125568313524 19 0.00725556723773 20 0.0174901075661 21 0.0180223416537 22 0.000814012193587 23 0.0033499461133 24 0.00398892955855 25 0.00144663790707 26 0.00152658030856 27 0.0425674766302 28 0.0102258082479 29 0.0018127564108 30 0.000755835149903 31 0.00456942431629 32 0.00314804422669 33 0.0249033384025 34 0.00119537930004 35 0.00245238281786 36 0.000161521922564 37 0.0120999542996 38 0.00182112539187 39 0.000555641890969 40 0.0217830613256 41 0.000875626225024 42 0.0151101183146 43 0.000738968024962 44 0.00464019179344 45 0.000326958775986 46 0.0020466952119 47 0.0027880910784 48 0.00334097701125 49 0.00112884491682 50 0.00240959180519

In [37]:
metrics.accuracy_score(y_val, predict(X_val))

0.99261904761904762

In [38]:
#prediction (In batches)
test = pd.read_csv('test.csv').dropna().values
test = test.reshape(-1,1,28,28).astype(theano.config.floatX)
test = test/255
np.savetxt('submission.csv', np.c_[np.arange(1,len(test) + 1),predict(test)], header = 'ImageId,Label', delimiter=',', comments = '', fmt='%d')

In [2]:
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

[GpuElemwise{exp,no_inplace}(<CudaNdarrayType(float32, vector)>), HostFromGpu(GpuElemwise{exp,no_inplace}.0)]
Looping 1000 times took 0.788831 seconds
Result is [ 1.23178029  1.61879349  1.52278066 ...,  2.20771813  2.29967761
  1.62323296]
Used the gpu
